Make sure to select the conda base kernel

In [12]:
import sys
print(sys.executable)
!conda list

/opt/conda/bin/python
# packages in environment at /opt/conda:
#
# Name                    Version                   Build  Channel
_libgcc_mutex             0.1                        main  
_openmp_mutex             5.1                       1_gnu  
archspec                  0.2.1              pyhd3eb1b0_0  
asttokens                 2.0.5              pyhd3eb1b0_0  
boltons                   23.0.0          py311h06a4308_0  
brotli-python             1.0.9           py311h6a678d5_7  
bzip2                     1.0.8                h7b6447c_0  
c-ares                    1.19.1               h5eee18b_0  
ca-certificates           2023.12.12           h06a4308_0  
certifi                   2023.11.17      py311h06a4308_0  
cffi                      1.16.0          py311h5eee18b_0  
charset-normalizer        2.0.4              pyhd3eb1b0_0  
comm                      0.1.2           py311h06a4308_0  
conda                     23.11.0         py311h06a4308_0  
conda-libmamba-solver     23

In [6]:
# !pip install scikit-image
# !pip install numpy

In [7]:
from skimage import io
import numpy as np
import os

In [8]:
os.getcwd()

'/workspaces/MSSR-Statin-Project/scripts'

In [9]:
# filenames are structured with '_w1', '_w2', '_w3' to indicate channels.
directory = '../images/to_combine'
output_directory = '../images'

In [10]:

# Create lists for different wavelength images
w1_images, w2_images, w3_images = [], [], []

# Populate the lists with corresponding image filenames
for filename in sorted(os.listdir(directory)):
    if ("_w1" in filename) and ('._helix' not in filename):
        w1_images.append(filename)
    elif ("_w2" in filename) and ('._helix' not in filename):
        w2_images.append(filename)
    elif ("_w3" in filename) and ('._helix' not in filename):
        w3_images.append(filename)

In [11]:
for w1, w2, w3 in zip(w1_images, w2_images, w3_images):
    # Read the images
    img_w1 = io.imread(os.path.join(directory, w1), as_gray=True)
    img_w2 = io.imread(os.path.join(directory, w2), as_gray=True)
    img_w3 = io.imread(os.path.join(directory, w3), as_gray=True)

    # Stack the images along the third axis to create an RGB image
    # Note: Ensure that the images are correctly scaled (0-1 range for floats, 0-255 for integers)
    img_w1 = img_w1 / img_w1.max()
    img_w2 = img_w2 / img_w2.max()
    img_w3 = img_w3 / img_w3.max()

    rgb_image = np.stack((img_w3, img_w2, img_w1), axis=-1)
    
    # Assume w1 is something like 'image1.tif'
    base_filename = os.path.splitext(w1)[0]  # This will remove the .tif and give 'image1'    
    output_filename = 'RGB_{}.tif'.format(base_filename)
    # Save or display the image
    io.imsave(os.path.join(output_directory, output_filename), (rgb_image * 255).astype(np.uint8))
    print(output_filename)
    # io.imshow(rgb_image)  # Display the image

RGB_helix-20180720-10-NIH1_B21_w1.tif
RGB_helix-20180720-10-NIH1_I18_w1.tif
